In [1]:
import pandas as pd
import numpy as np

import nltk
nltk.download('movie_reviews')
from nltk.corpus import movie_reviews as mr

# tokenizers
from nltk.tokenize import word_tokenize, sent_tokenize, RegexpTokenizer
nltk.download(['punkt', 'stopwords', 'wordnet'])
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer, SnowballStemmer, LancasterStemmer


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline

# !pip install afinn -q
# from afinn import Afinn
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.linear_model import LogisticRegression
nltk.download('vader_lexicon')

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


# Data

In [2]:
# This is a movie reviews dataset which contains the reviews to the moviews
# and the ground true label indicating whether the review is postive or negative.
# In this report, we will draw a portion of the review data for analysis
documents = [(' '.join(list(mr.words(fileid))), category)
             for category in mr.categories()
             for fileid in mr.fileids(category)[:100]]
df = pd.DataFrame(documents, columns=['text', 'label'])

In [3]:
df.head()

text label
0  plot : two teen couples go to a church party ,...   neg
1  the happy bastard ' s quick movie review damn ...   neg
2  it is movies like these that make a jaded movi...   neg
3  " quest for camelot " is warner bros . ' first...   neg
4  synopsis : a mentally unstable man undergoing ...   neg

## Feature Engineering

### Bag of words

Bag of Words (BoW) is a common method that converts text data into numerical vectors that machine learning algorithms can understand (it is an important basis further feature engineering and analysis). It represents text as a collection of individual words without considering order or grammar, but preserving the frequency of each word.

To generate the BoW, we first create a vocabulary of unique words from all documents. Then, each document is turned into a vector, with values representing the frequency of each vocabulary word within the document.

For example, consider the sentences: "The cat is blue", "The dog is purple.". Then the vocabulary is :{The, cat, is, blue, purple, dog}. And the sentences can be represented in the numerical form: [1,1,1,1,0,0] and [1,0,1,0,1,1]

In this example, (1)we create the BoW based on the raw text and use it to train a Naive Bays model to predict the review labels; (2) a tf.idf vector is generated based on the BoW, and a random forest model is trained on the tf.idf vector. In subsequent sections, we also present that using the same machine learning models on preprocessed text data can lead to improved performance.



In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


In [5]:
# split the text data into train test set
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], random_state=1)

In [6]:
vectorizer = CountVectorizer()
#fit and transform the training data to BoW
X_train_bow = vectorizer.fit_transform(X_train)
# convert the test data to BoW
X_test_bow = vectorizer.transform(X_test)

In [7]:
# create a MultinomialNB and fit to the training data
nb = MultinomialNB()
nb.fit(X_train_bow, y_train)

# test the model on test dataset
y_pred = nb.predict(X_test_bow)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

         neg       0.62      0.56      0.59        27
         pos       0.54      0.61      0.57        23

    accuracy                           0.58        50
   macro avg       0.58      0.58      0.58        50
weighted avg       0.59      0.58      0.58        50



In [8]:
# create a TfidfVectorizer to build the TF-IDF matrix based on the preprocessed data
# in this example, we split words just by word boundries
vectorizer = TfidfVectorizer(lowercase=False, token_pattern=r'\b\w+\b')
# fit to the training data  and test data and transform them to TF-IDF
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)
# create the rf model
clf = RandomForestClassifier(random_state=22)
# create piple line for the vecotirzer and model for cv
pipeline1 = make_pipeline(vectorizer, clf)

# perform 5-fold CV to get the avg scores
scores1 = cross_val_score(pipeline1, X_train, y_train, cv=5, scoring='accuracy')
print(f"RF model avg accuracy on raw text: {scores1.mean():.4f} (+/- {scores1.std()*2:.4f})")

clf.fit(X_train_tfidf, y_train)
y_pred = clf.predict(X_test_tfidf)
#classification report
print(classification_report(y_test, y_pred))

RF model avg accuracy on raw text: 0.6600 (+/- 0.1147)
              precision    recall  f1-score   support

         neg       0.79      0.56      0.65        27
         pos       0.61      0.83      0.70        23

    accuracy                           0.68        50
   macro avg       0.70      0.69      0.68        50
weighted avg       0.71      0.68      0.68        50



### Tokenization, stemming, lemmatization, and other preprocessing

#### Tokenization

Tokenization is the process of breaking down input text into smaller pieces, referred to as tokens. Within text processing, a token is most often a word. However, it can also represent characters, subwords, or even entire sentences.

There are three primary methods to achieve tokenization:

(1) Word Tokenizer: This method divides text into individual words.
(2) Sentence Tokenizer: This method breaks down text into distinct sentences.
(3) Regular Expression Tokenizer: This method allows users to split text based on custom patterns.

Within the NLTK package, both word and sentence tokenization are based on the Punkt tokenizer model, which is a pre-trained unsupervised machine learning model. It uses punctuation and capitalization cues to identify the start and end of sentences. And further split the sentences into words.
reference:
Unsupervised Multilingual Sentence Boundary Detection (Kiss and Strunk, 2005)


In this section, we provide example of using word/sentence/customized regex tokenizer to split raw text into our desired tokens.

In [9]:
# (1) word tokenization takes the raw text and tokenize them into words. Note that
# the puncatuations are also treated as a word
df['word_tokens'] = df['text'].apply(word_tokenize)
df['word_tokens'].head(2).tolist()

[['plot',
  ':',
  'two',
  'teen',
  'couples',
  'go',
  'to',
  'a',
  'church',
  'party',
  ',',
  'drink',
  'and',
  'then',
  'drive',
  '.',
  'they',
  'get',
  'into',
  'an',
  'accident',
  '.',
  'one',
  'of',
  'the',
  'guys',
  'dies',
  ',',
  'but',
  'his',
  'girlfriend',
  'continues',
  'to',
  'see',
  'him',
  'in',
  'her',
  'life',
  ',',
  'and',
  'has',
  'nightmares',
  '.',
  'what',
  "'",
  's',
  'the',
  'deal',
  '?',
  'watch',
  'the',
  'movie',
  'and',
  '``',
  'sorta',
  '``',
  'find',
  'out',
  '.',
  '.',
  '.',
  'critique',
  ':',
  'a',
  'mind',
  '-',
  'fuck',
  'movie',
  'for',
  'the',
  'teen',
  'generation',
  'that',
  'touches',
  'on',
  'a',
  'very',
  'cool',
  'idea',
  ',',
  'but',
  'presents',
  'it',
  'in',
  'a',
  'very',
  'bad',
  'package',
  '.',
  'which',
  'is',
  'what',
  'makes',
  'this',
  'review',
  'an',
  'even',
  'harder',
  'one',
  'to',
  'write',
  ',',
  'since',
  'i',
  'generally',
  

In [10]:
# However, it is hard for the basic tokenizers to detect all
# the boundaries correctly, for example, it may treat abbrivations and punctuations
# inside a sentence as a sentence boundry, for example, Ph.d., dr. who. And some people use
# capital letters to express their emotions, the sentence tokenizer may take that
# as the initial of each sentence. Hence, it's necessary to use regular expression
# based tokenizer, which allows us to customize the splitting pattern.

df['sent_tokens'] = df['text'].apply(sent_tokenize)
df['sent_tokens'].head(2).tolist()

[['plot : two teen couples go to a church party , drink and then drive .',
  'they get into an accident .',
  'one of the guys dies , but his girlfriend continues to see him in her life , and has nightmares .',
  "what ' s the deal ?",
  'watch the movie and " sorta " find out .',
  '.',
  '.',
  'critique : a mind - fuck movie for the teen generation that touches on a very cool idea , but presents it in a very bad package .',
  "which is what makes this review an even harder one to write , since i generally applaud films which attempt to break the mold , mess with your head and such ( lost highway & memento ) , but there are good and bad ways of making all types of films , and these folks just didn ' t snag this one correctly .",
  'they seem to have taken this pretty neat concept , but executed it terribly .',
  'so what are the problems with the movie ?',
  "well , its main problem is that it ' s simply too jumbled .",
  'it starts off " normal " but then downshifts into this " fant

In [11]:
print(f"output={sent_tokenize('The cat is blue. The dog is purple.')}")
print(f"output={word_tokenize('The cat is blue. The dog is purple.')}")

output=['The cat is blue.', 'The dog is purple.']
output=['The', 'cat', 'is', 'blue', '.', 'The', 'dog', 'is', 'purple', '.']


In [12]:
# (3) regular expression based tokenizer. In this example, we use a customize pattern
# that tokenizes the text into words.
tokenizer = RegexpTokenizer("\w+|\$[\d\.]+|\S+")
df['regex_tokens'] = df['text'].apply(RegexpTokenizer("\w+|\$[\d\.]+|\S+").tokenize)
df['regex_tokens'].head(2).tolist()

[['plot',
  ':',
  'two',
  'teen',
  'couples',
  'go',
  'to',
  'a',
  'church',
  'party',
  ',',
  'drink',
  'and',
  'then',
  'drive',
  '.',
  'they',
  'get',
  'into',
  'an',
  'accident',
  '.',
  'one',
  'of',
  'the',
  'guys',
  'dies',
  ',',
  'but',
  'his',
  'girlfriend',
  'continues',
  'to',
  'see',
  'him',
  'in',
  'her',
  'life',
  ',',
  'and',
  'has',
  'nightmares',
  '.',
  'what',
  "'",
  's',
  'the',
  'deal',
  '?',
  'watch',
  'the',
  'movie',
  'and',
  '"',
  'sorta',
  '"',
  'find',
  'out',
  '.',
  '.',
  '.',
  'critique',
  ':',
  'a',
  'mind',
  '-',
  'fuck',
  'movie',
  'for',
  'the',
  'teen',
  'generation',
  'that',
  'touches',
  'on',
  'a',
  'very',
  'cool',
  'idea',
  ',',
  'but',
  'presents',
  'it',
  'in',
  'a',
  'very',
  'bad',
  'package',
  '.',
  'which',
  'is',
  'what',
  'makes',
  'this',
  'review',
  'an',
  'even',
  'harder',
  'one',
  'to',
  'write',
  ',',
  'since',
  'i',
  'generally',
  'a

In [13]:
# Using this customized pattern, we can find that regex tokenizer recognize the "$20.50"
# as an complete entity, while the word tokenizer splited the character and floats
tokenizer = RegexpTokenizer("\w+|\$[\d\.]+|\S+")
text = "I bought 3 apples for $20.50. That's too expensive."
print(f"word tokenizer: {word_tokenize(text)}\nsentence tokenzer: {sent_tokenize(text)}\nregex tokenizer:{tokenizer.tokenize(text)}")


word tokenizer: ['I', 'bought', '3', 'apples', 'for', '$', '20.50', '.', 'That', "'s", 'too', 'expensive', '.']
sentence tokenzer: ['I bought 3 apples for $20.50.', "That's too expensive."]
regex tokenizer:['I', 'bought', '3', 'apples', 'for', '$20.50.', 'That', "'s", 'too', 'expensive', '.']


#### lemmatization & stemming

Both stemming and lemmatization are text normalization processes that reduce words to their base or root form. This is very critical in text data analysis, as a word can have many forms but still convey the same meaning. When we perform the analysis, we want to ensure that derivations of a word are treated as a single item. For example, we might want to consider "computing", "computed", and "computation" as the same term, "comput". This ensures consistency in representation, prevent overfitting to a specific term, and also reduce the dimension/number of unique words in the dataset.

However, the two methods have their differences. Stemming typically works by simply chopping off the affixes from words, while lemmatization uses a dictionary and takes into account the morphological analysis of the words to return words to their base form.
For example, considering the word "flies", Porter Stemmer would convert it to "fli", while a WordNet Lemmatizer would convert it to "fly".

Lowercasing is the process of converting all the words to lowercase to ensure consistency. Additionally, removing stop words involves using a predefined list of common words and eliminating them from the text.

There are many popular lemmatizer and stemmers including wordnet lemmatizer, snowball stemmer, and lancaster stemmer. In our example, we analyze the effects of each stemmer/lemmatizer and other normalization techniques like lowercasing and removing stop words. We then build a Naive Bayes model and show that these text preprocessing could improves the model's performance.



In [14]:
text_tool={'WordNetLemmatizer':WordNetLemmatizer().lemmatize,
           'SnowballStemmer':SnowballStemmer('english').stem,
           'LancasterStemmer':LancasterStemmer().stem}

In [15]:
# convert tokens to lowercase
df['regex_tokens'] = df['regex_tokens'].apply(lambda tokens: [token.lower() for token in tokens])
# remove stopwords (this is not implemented as it dose improve the model's performace)
stop_words = set(stopwords.words('english'))
# df['regex_tokens'] = df['regex_tokens'].apply(lambda tokens: [token for token in tokens if token not in stop_words])
# Apply lemmatization, in this case the wordnet lemmatizer is used
df['regex_tokens'] = df['regex_tokens'].apply(lambda tokens: [text_tool['WordNetLemmatizer'](token) for token in tokens])

In [16]:
# concate the words to complete sentence for further analysis
df['regex_tokens_str'] = df['regex_tokens'].apply(' '.join)
X_train, X_test, y_train, y_test = train_test_split(df['regex_tokens_str'], df['label'], random_state=1)

In [17]:
# bag of words
vectorizer = CountVectorizer()
# Fit to the training data and transform it to BoW
X_train_bow = vectorizer.fit_transform(X_train)
# Transform the test data to BoW
X_test_bow = vectorizer.transform(X_test)

# Create a MultinomialNB model
nb = MultinomialNB()
# Fit to the training data
nb.fit(X_train_bow, y_train)

# test the model on test dataset
y_pred = nb.predict(X_test_bow)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         neg       0.68      0.56      0.61        27
         pos       0.57      0.70      0.63        23

    accuracy                           0.62        50
   macro avg       0.63      0.63      0.62        50
weighted avg       0.63      0.62      0.62        50



### TF-IDF

TF-IDF is a numerical statistic that reflects how important a word is to a document in a collection or corpus. It is composed by two parts, Term Frequency (TF) and Inverse Document Frequency (IDF).
TF represents the frequency of a term in a document. It's the ratio of (the number of times the term appears in a document) to (the total number of terms in the document).
IDF Measures the importance of a term. It's calculated as the logarithm of (the total number of documents ->N) divided by the (number of documents containing the term).


In Machine learning, models like randomforest cannot directly process text data, TF-IDF can be used to transform raw text data into a set of numerical features for modeling. It is also very helpful for search engines and information retrieveal systems, as it can rank documents by their relevance to a query term or terms.

In our example, we covereted the preprocessed text to it's TF.IDF form, and then implemented a random forest model on it to predict the label of moview reviews.

In [18]:
# create a TfidfVectorizer to build the TF-IDF matrix based on the text data
# note that the train test data are from previous section for preprocessing [3]
vectorizer = TfidfVectorizer(token_pattern=r'\w+|\$[\d\.]+|\S+')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# create the rf model
clf = RandomForestClassifier(random_state=22)
pipeline = make_pipeline(vectorizer, clf)

# perform 5-fold CV to get the avg scores
scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='accuracy')
print(f"RF model 5-fold cv avg accuracy on preprocessed text: {scores.mean():.4f} (+/- {scores.std()*2:.4f})")

clf.fit(X_train_tfidf, y_train)
y_pred = clf.predict(X_test_tfidf)

#classification report
print(classification_report(y_test, y_pred))

RF model 5-fold cv avg accuracy on preprocessed text: 0.7200 (+/- 0.1611)
              precision    recall  f1-score   support

         neg       0.80      0.74      0.77        27
         pos       0.72      0.78      0.75        23

    accuracy                           0.76        50
   macro avg       0.76      0.76      0.76        50
weighted avg       0.76      0.76      0.76        50



### sentiment analysis

In [19]:
# Sentiment analysis is the process of determing the emotional tone or subjective opinion
# within text data. The primary methods for sentiment analysis is:
# (1) lexicon-based: this method use a dictionary of sentiment socres for words.
# When a sentence is inputed, it calculate a general score based on the score
# obtained by certain word.
# For a simple example, given the lexicon: {"good" : +1, "bad" : -1,"mediocore" : 0}
# Then "the movie is very good." will recieve a sentiment score of 1
# (2) machine learning or deep learning based: Models like naive bays, random forest
# , and neural networks can be trained to predict the emotion of the text. The
# features are usually in the vecoterized form like  BoW or TF-IDF as we previously mentioned.

# In this example, we will use Vader (Valence Aware Dictionary and sEntiment Reasoner).
# Vader is a lexicon and rule-based sentiment analysis tool.It is specifically designed
# for sentiment analysis of social media texts and is capable of understanding
# the context of words, such as intensifiers or diminishers, to provide accurate sentiment scores.

# Vader provides a compound score in the range of 1 or -1 which represents the overall
# sentiment of a text. Positive values indicate positive sentiments, while negative
# values suggest negative sentiments.

# For our dataset, we'll calculate the Vader sentiment score for each text entry.
# This score will then serve as a feature for our logistic regression model to classify
# the movie reivews as "pos" or "neg".

## reference:
# Hutto, C., & Gilbert, E. (2014). VADER: A Parsimonious Rule-Based Model for
# Sentiment Analysis of Social Media Text. Proceedings of the International AAAI
# Conference on Web and Social Media, 8(1), 216-225. https://doi.org/10.1609/icwsm.v8i1.14550

In [20]:
# calcuate the sentiment score for the text
sia = SentimentIntensityAnalyzer()
df['sentiment_score'] = df['regex_tokens_str'].apply(lambda text: sia.polarity_scores(text)['compound'])
df[['text','sentiment_score']].head()

text  sentiment_score
0  plot : two teen couples go to a church party ,...           0.9919
1  the happy bastard ' s quick movie review damn ...           0.8938
2  it is movies like these that make a jaded movi...           0.9820
3  " quest for camelot " is warner bros . ' first...          -0.4537
4  synopsis : a mentally unstable man undergoing ...           0.9259

In [21]:
# Sort the DataFrame by sentiment_score in descending order for top 3 and tail3
# From the results, we can see that the vader score matches the labels. And it
# captures the emotions effectively.
sorted_df = df.sort_values(by='sentiment_score', ascending=False)

In [22]:
for index, row in sorted_df.head(3).iterrows():
    print("Text:", row['text'])
    print(f"Sentiment Score: {row['sentiment_score']}  - label: {row['label']}")
    print('-' * 50)  # print separator for clarity

Text: the most common ( and in many cases the only ) complaint against francis ford coppola ' s 1972 masterpiece the godfather is glamorising of mafia , which is presented as an institution guided by ancient tradition and virtues like honour , loyalty and solidarity more suitable for some gentler , kinder ages . martin scorsese , another italoamerican moviemaker , confronted that perspective with his own , more down - to - earth vision of mafia in 1973 mean streets , movie that dealt with lower echelons of organised crime . unfortunately for scorsese , his film was unspectacular and too artsy to compete with coppola ' s influence on mafia portrayals in the movies . seventeen years later scorsese returned to mean streets of new york with another film that dealt with darker side of american organised crime . this film was goodfellas , epic black comedy which is today considered as one of the best and most influential films of 1990s . goodfellas owes some of its initial success and popula

In [23]:
for index, row in sorted_df.tail(3).iterrows():
    print("Text:", row['text'])
    print(f"Sentiment Score: {row['sentiment_score']}  - label: {row['label']}")
    print('-' * 50)  # print separator for clarity

Text: " first rule of fight club is , don ' t talk about fight club . " this quote deals directly with my opinion of this film , because quite frankly , it ' s not even worth mentioning . director david fincher ( seven ) has completely gone over the edge . with crude humor , an intolerable script and 2 hours and 19 minutes of pure torture , i ' m surprised i didn ' t walk out . this just goes to show that hype kills . it ' s very rare that a film so hyped up as " fight club " turns out to be noteworthy , and this is no exception . the story goes like this . edward norton is an insomniac trying to enlighten his dull life by going to a variety of disease therapy sessions , until he meets marla ( helena bonham carter ) . norton claims that she is wrecking his pleasurable lifestyle , when he spots her at a session that he also attended . apparently the disease sessions help them see what life is like , literally living in pain . why any person would think of such a thing is beyond me , but

In [24]:
# A logistic regression model is applied to predict sentiments based solely on the sentiment score.
# However, its performance is not as strong compared to other models implemented earlier.
# The is due to the limited feature set derived from the data. The model is not
# complecated enough to capture the pattern in the actual text data.
# And it is interesting to notice that the precision for "neg" review is very high
# while the recall is relatively very low. This means that the model is very conservative
# on making predicting positives and tends to make less False Positive errors.
# The precision and recall for the "pos" reviews are opposite to the ones for "neg".

# convert the label from string to integer
df['label_num'] = df['label'].map({'pos': 1, 'neg': 0})
# split the dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(df[['sentiment_score']], df['label_num'], random_state=1)
# create the model and train the modle on the training set
model = LogisticRegression()
model.fit(X_train, y_train)
# Make predictions on the testing set
# y_pred = model.predict(X_test)
probs = model.predict_proba(X_test)[:, 1]
# set the threshold threshold
y_pred = [1 if prob > 0.57 else 0 for prob in probs]

# Print the classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.37      0.53        27
           1       0.56      0.96      0.71        23

    accuracy                           0.64        50
   macro avg       0.74      0.66      0.62        50
weighted avg       0.75      0.64      0.61        50



### Text Sequencing and Padding

In [25]:
# In many NLP tasks (i.e. RNNs, LSTMs), raw text cannot be processed directly by algorithms. Instead, the text
# must be tokenized (split into words or subwords) and then translated into a sequence of integers.
# Text sequencing is the process of converting text data into sequences of integers.
# Each integer corresponds to a specific word or token in the text. The primary purpose
# of text sequencing is to transform textual data into a format that can be fed into machine learning models.

# Common approaches for text sequencing involve using a pre-defined vocabulary and assigning
# each word in the vocabulary a unique integer.

# For example, "I love MIE1624" is converted to [1,2,3] based on the dictionary
# vocab = {"I": 1, "love": 2, "MIE1624": 3, "MIE": 4, "UofT": 0}
# sequence -> [1,2,3]


# Sequence padding is a preprocessing step used to ensure that all sequences in a dataset have the same length.
# This is necessary because many machine learning models like Recurrent Neural Networks (RNNs) and
# Long Short-Term Memory networks (LSTMs), require input sequences to be of the same length.

# The process includes two essential parts:
# (1) Padding shorter sequences: If a sequence is shorter than the desired length,
# zeros (or a specified padding value) are added to the sequence to achieve the required length.
# (2) Truncating longer sequences: If a sequence is longer than the desired length,
# it is truncated to fit the required length.

In [26]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df['regex_tokens_str'])
sequences = tokenizer.texts_to_sequences(df['regex_tokens_str'])
# padding the sequence
data = pad_sequences(sequences, maxlen=100)


In [27]:
X_train, X_temp, y_train, y_temp = train_test_split(data, df['label_num'], test_size=0.3, random_state=1)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.6667, random_state=1)



In [28]:
# Define a simple example LSTM model [3]
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=64))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

# Compile the model with a smaller learning rate
opt = Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [29]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5)


Epoch 1/5
5/5 [==============================] - 7s 519ms/step - loss: 0.6935 - accuracy: 0.4857 - val_loss: 0.6916 - val_accuracy: 0.5789
Epoch 2/5
5/5 [==============================] - 2s 374ms/step - loss: 0.6900 - accuracy: 0.5143 - val_loss: 0.6914 - val_accuracy: 0.5263
Epoch 3/5
5/5 [==============================] - 2s 403ms/step - loss: 0.6786 - accuracy: 0.8429 - val_loss: 0.6965 - val_accuracy: 0.5263
Epoch 4/5
5/5 [==============================] - 3s 503ms/step - loss: 0.6066 - accuracy: 0.8714 - val_loss: 0.6655 - val_accuracy: 0.6316
Epoch 5/5
5/5 [==============================] - 2s 373ms/step - loss: 0.3904 - accuracy: 0.8786 - val_loss: 0.8458 - val_accuracy: 0.4737


In [30]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

2/2 [==============================] - 0s 44ms/step - loss: 0.7875 - accuracy: 0.5610
Test Loss: 0.7875228524208069
Test Accuracy: 0.5609756112098694


# Reference

In [31]:
# 1. Hutto, C., & Gilbert, E. (2014). VADER: A Parsimonious Rule-Based Model for
# Sentiment Analysis of Social Media Text. Proceedings of the International AAAI
# Conference on Web and Social Media, 8(1), 216-225. https://doi.org/10.1609/icwsm.v8i1.14550

# 2. Kiss, T., & Strunk, J. (2006). Unsupervised Multilingual Sentence Boundary
# Detection. Computational Linguistics, 32, 485-525.

# 3. ChatGPT. (2023, October 24).

